In [16]:
from mmcv.utils import Config
from mmseg.apis import train_segmentor
from mmseg.models import build_segmentor
from mmseg.datasets import build_dataset, build_dataloader
from mmseg.utils import build_dp, setup_multi_processes

config = r'configs/fcn/fcn_r18b-d8_512x1024_20k_chestxray.py'



cfg = Config.fromfile(config)
cfg.gpu_ids = range(1)
cfg.seed = 255

setup_multi_processes(cfg)

model = build_segmentor(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()

dataset = [build_dataset(cfg.data.train)]

distributed = False
validate = True
#train_segmentor(
#    model,
#    datasets,
#    cfg,
#    distributed=False,
#    validate=True)




2022-11-19 11:20:38,007 - mmseg - INFO - Multi-processing start method is `None`
2022-11-19 11:20:38,008 - mmseg - INFO - OpenCV num_threads is `8
2022-11-19 11:20:38,171 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnet50_v1c'}
2022-11-19 11:20:38,171 - mmcv - INFO - load model from: open-mmlab://resnet50_v1c
2022-11-19 11:20:38,172 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnet50_v1c
2022-11-19 11:20:38,245 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for layer1.0.conv1.weight: copying a param with shape torch.Size([64, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
size mismatch for layer1.1.conv1.weight: copying a param with shape torch.Size([64, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
size mismatch for layer2.0.conv1.weight: copying a param with shape torch.Size([128, 256, 1

In [18]:

# The default loader config
loader_cfg = dict(
    # cfg.gpus will be ignored if distributed
    num_gpus=len(cfg.gpu_ids),
    dist=distributed,
    seed=cfg.seed,
    drop_last=True)
# The overall dataloader settings
loader_cfg.update({
    k: v
    for k, v in cfg.data.items() if k not in [
        'train', 'val', 'test', 'train_dataloader', 'val_dataloader',
        'test_dataloader'
    ]
})
# The specific dataloader settings
train_loader_cfg = {**loader_cfg, **cfg.data.get('train_dataloader', {})}
dataset, train_loader_cfg

([<mmseg.datasets.lungseg.LungSegmentationDataset at 0x7f6a9c8540d0>],
 {'num_gpus': 1,
  'dist': False,
  'seed': 255,
  'drop_last': True,
  'samples_per_gpu': 2,
  'workers_per_gpu': 2})

In [14]:
from torch.utils.data import DataLoader, IterableDataset

dl = DataLoader(dataset)
print(len(dl))
next(iter(dl))

1


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'mmseg.datasets.lungseg.LungSegmentationDataset'>

In [19]:
data_loaders = [build_dataloader(ds, **train_loader_cfg) for ds in dataset] # build PyTorch data loader
data_loaders = [build_dataloader(dataset, **train_loader_cfg)] # build PyTorch data loader

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
next(data_loaders[0])

: 